# 数据结构的内置方法

这一节介绍pandas数据结构内置的常用方法。很重要的一节。

创建本节要用到的数据结构。

In [1]:
import numpy as np
import pandas as pd

In [2]:
index  = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])
wp = pd.Panel(np.random.randn(2,5,4), items=['Item1', 'Item2'], major_axis=pd.date_range('1/1/2000',periods=5),
             minor_axis=['A', 'B', 'C', 'D'])

## Head() Tail()

想要预览Series或DataFrame对象，可以使用head()和tail()方法。默认显示的行数是5，你也可以自己设置显示的行数。

In [8]:
long_series = pd.Series(np.random.randn(1000))

In [9]:
long_series.head()

0   -0.229733
1    0.355929
2   -0.016854
3    0.790954
4   -1.099537
dtype: float64

In [10]:
long_series.tail(3)

997   -1.717307
998   -0.104273
999    0.377712
dtype: float64

## 属性和 numpy数组

pandas对象有很多属性，你可以通过这些属性访问数据。

* **shape**: 显示对象的维度，同ndarray
* 坐标label
* * **Series**: 索引
* * **DataFrame**:索引(行)和列
* * **Panel**:  items, major_axis and minor_axis

这些属性都可以安全滴进行赋值！

In [11]:
df[:2]

,A,B,C
2000-01-01,0.961850,0.832213,0.698852
2000-01-02,0.945025,-0.367759,0.061104


In [14]:
df.columns = [x.lower() for x in df.columns] #将列名重置为小写

In [15]:
df

,a,b,c
2000-01-01,0.961850,0.832213,0.698852
2000-01-02,0.945025,-0.367759,0.061104
2000-01-03,1.341735,-0.520314,0.621666
2000-01-04,-0.749680,0.737349,2.863382
2000-01-05,-0.395510,-1.115914,-0.739005
2000-01-06,0.868806,1.731622,-1.167939
2000-01-07,-2.260882,-0.748856,-1.483881
2000-01-08,0.002094,-0.744713,0.701304


只想得到对象中的数据而忽略列名索引这些乱七八糟的，使用values属性就可以

In [17]:
s.values

array([-1.0891997 ,  2.14039355, -0.42522808, -0.99005429,  0.44079549])

In [19]:
df.values

array([[  9.61849648e-01,   8.32213212e-01,   6.98851835e-01],
       [  9.45024544e-01,  -3.67758714e-01,   6.11041983e-02],
       [  1.34173499e+00,  -5.20313703e-01,   6.21666043e-01],
       [ -7.49680310e-01,   7.37349183e-01,   2.86338159e+00],
       [ -3.95509934e-01,  -1.11591412e+00,  -7.39005372e-01],
       [  8.68805636e-01,   1.73162237e+00,  -1.16793946e+00],
       [ -2.26088188e+00,  -7.48856000e-01,  -1.48388067e+00],
       [  2.09428537e-03,  -7.44713066e-01,   7.01303507e-01]])

In [20]:
type(df.values)

numpy.ndarray

In [21]:
wp.values

array([[[-1.70028729,  0.0228584 , -1.31088058, -1.81587031],
        [ 0.97998246, -0.91700322, -0.61644728,  0.10122091],
        [-0.0380311 ,  0.42598951,  0.43708298,  1.4847927 ],
        [ 0.11626489, -0.82049708, -0.3390273 , -0.64795144],
        [ 1.69714832,  3.5289194 , -1.04279508, -0.70495814]],

       [[-1.67817219,  0.12514369,  0.68422357,  0.08062269],
        [ 1.06218021, -1.0658129 ,  1.20032917, -0.02492852],
        [-1.46904051, -0.94881081, -0.62609046, -0.86555444],
        [-1.90217969,  0.3482584 ,  0.55615558,  0.76624635],
        [-0.46576376, -0.52609712,  0.46101983, -0.4530962 ]]])

如果DataFrame或Panel对象的数据类型相同(比如都是 int64)，修改object.values相当于直接修改原对象的值。如果数据类型不相同，则根本不能对values属性返回值进行赋值。

**注意**:

如果对象内数据类型不同，values返回的ndarray的dtype将是能够兼容所有数据类的类型。比如，数据是int和float，dtype将是float。

## 提高运算速度

pandas从0.11.0版本开始使用numexpr库对二值数值类型操作加速，用bottleneck库对布尔操作加速。

加速效果对大数据尤其明显。

这里有一个速度的简单对比，使用100,000行\* 100列的DataFrame:

![](https://ooo.0o0.ooo/2016/04/14/570f692c2e8a1.png)

所以，在安装pandas后也要顺便安装numexpr, bottleneck。

## 灵活的二元运算

在所有的pandas对象之间的二元运算中，大家最感兴趣的一般是下面两个：
* 广播(broadcasting)
* 缺失值计算

二者也可以结合使用，我们下面一一为大家介绍：

### 广播

DataFrame对象内置add(),sub(),mul(),div()以及radd(), rsub(),...等方法。

至于广播计算，Series的输入是最有意思的。

In [67]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

In [68]:
df

,one,three,two
a,-0.819889,NaN,-0.748560
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [69]:
row = df.ix[1]
row

one      1.206005
three   -0.586788
two      0.630745
Name: b, dtype: float64

In [70]:
column = df['two']

In [71]:
column

a   -0.748560
b    0.630745
c   -0.227499
d   -1.590267
Name: two, dtype: float64

In [72]:
df.sub(row, axis='columns')

,one,three,two
a,-2.025895,NaN,-1.379305
b,0.000000,0.000000,0.000000
c,-1.826850,0.506291,-0.858244
d,NaN,-0.908320,-2.221013


In [73]:
df.sub(row, axis=1)

,one,three,two
a,-2.025895,NaN,-1.379305
b,0.000000,0.000000,0.000000
c,-1.826850,0.506291,-0.858244
d,NaN,-0.908320,-2.221013


In [74]:
df.sub(row, axis='index')

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


In [75]:
df.sub(row, axis=0)

,one,three,two
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN
one,NaN,NaN,NaN
three,NaN,NaN,NaN
two,NaN,NaN,NaN


### 填充缺失值

在Series和DataFrame中，算术运算方法(比如add())有一个fill_value参数，含义很明显，计算前用一个值来代替缺失值，然后再参与运算。注意，如果参与运算的两个object同一位置(同行同列)都是NaN，fill_value不起作用，计算结果还是NaN。

看例子：

In [76]:
df

,one,three,two
a,-0.819889,NaN,-0.748560
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [91]:
df2 = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                       'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                       'three' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd'])})

In [92]:
df2

,one,three,two
a,-0.745516,-0.657512,-1.358255
b,0.249448,-0.829815,0.258827
c,-0.117322,-0.016284,0.144683
d,NaN,0.213930,1.766173


In [93]:
df + df2

,one,three,two
a,-1.565406,NaN,-0.358255
b,1.455453,-1.416603,0.889572
c,-0.738167,-0.096782,-0.082815
d,NaN,-1.281178,0.175906


In [95]:
 df.add(df2, fill_value=0) #注意['a', 'three']不是NaN

,one,three,two
a,-1.565406,-0.657512,-0.358255
b,1.455453,-1.416603,0.889572
c,-0.738167,-0.096782,-0.082815
d,NaN,-1.281178,0.175906


### 灵活的比较操作

pandas引入了二元比较运算方法：eq, ne, lt, gt, le。

In [96]:
df.gt(df2)

,one,three,two
a,False,False,True
b,True,True,True
c,False,False,False
d,False,False,False


In [97]:
df2.ne(df)

,one,three,two
a,True,True,True
b,True,True,True
c,True,True,True
d,True,True,True


意思操作返回一个和输入对象同类型的对象，值类型为bool，返回结果可以用于检索。

### 布尔降维 Boolean Reductions

pandas提供了三个方法(any(), all(), bool())和一个empty属性来对布尔结果进行降维。

In [100]:
df>0

,one,three,two
a,False,False,True
b,True,False,True
c,False,False,False
d,False,False,False


In [101]:
(df>0).all() #与操作

one      False
three    False
two      False
dtype: bool

In [103]:
(df > 0).any()#或操作

one       True
three    False
two       True
dtype: bool

同样可以对降维后的结果再进行降维。

In [104]:
(df > 0).any().any()

True

使用empty属性检测一个pandas对象是否为空。

In [105]:
df.empty

False

In [106]:
pd.DataFrame(columns=list('ABC')).empty

True

对于只含有一个元素的pandas对象，对其进行布尔检测，使用bool():

In [108]:
pd.Series([True]).bool()

True

In [109]:
pd.Series([False]).bool()

False

In [111]:
pd.DataFrame([[True]]).bool()

True

In [112]:
pd.DataFrame([[False]]).bool()

False

### 比较对象是否相等

一个问题通常有多种解法。一个最简单的例子：df+df和df\*2。为了检测两个计算结果是否相等，你可能想到：(df+df == df\*2).all()，然而，这样计算得到的结果是False：

In [113]:
df + df == df*2

,one,three,two
a,True,False,True
b,True,True,True
c,True,True,True
d,False,True,True


In [114]:
(df+df == df*2).all()

one      False
three    False
two       True
dtype: bool

为什么df + df == df\*2 返回的结果含有False？因为NaN和NaN比较厚结果为False！

In [116]:
np.nan == np.nan

False

还好pandas提供了equals()方法解决上面NaN之间不想等的问题。

In [119]:
(df+df).equals(df*2)

True

**注意**：

在使用equals()方法进行比较时，两个对象如果数据不一致必为False。

In [120]:
df1 = pd.DataFrame({'c':['f',0,np.nan]})
df1

,c
0,f
1,0
2,NaN


In [121]:
df2 = pd.DataFrame({'c':[np.nan, 0, 'f']}, index=[2,1,0])
df2

,c
2,NaN
1,0
0,f


In [122]:
df1.equals(df2)

False

In [123]:
df1.equals(df2.sort_index()) #对df2的索引排序，然后再比较

True

### 不同类型的对象之间 逐元素比较

你可以直接对pandas对象和一个常量值进行逐元素比较：

In [124]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [125]:
pd.Index(['foo', 'bar', 'baz']) == 'foo'

array([ True, False, False], dtype=bool)

不同类型的对象(比如pandas数据结构、numpy数组)之间进行逐元素的比较也是没有问题的，**前提是两个对象的shape要相同**。

In [128]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [129]:
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [130]:
pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar']) #长度不相同

ValueError: Series lengths must match to compare

但要知道不同shape的numpy数组之间是可以直接比较的！因为广播！即使无法广播，也不会Error而是返回False。

In [133]:
np.array([1,2,3]) == np.array([2])

array([False,  True, False], dtype=bool)

In [134]:
np.array([1, 2, 3]) == np.array([1, 2])

c:\python27\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


False

###  combine_first()

看一下例子：

In [135]:
df1 = pd.DataFrame({'A' : [1., np.nan, 3., 5., np.nan],
                    'B' : [np.nan, 2., 3., np.nan, 6.]})
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [136]:
df2 = pd.DataFrame({'A' : [5., 2., 4., np.nan, 3., 7.],
                        'B' : [np.nan, np.nan, 3., 4., 6., 8.]})
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [137]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


解释：

对于df1中NaN的元素，用df2中对应位置的元素替换！

### DataFrame.combine()

DataFrame.combine()方法接收一个DF对象和一个combiner方法。

In [140]:
combiner = lambda x,y: np.where(pd.isnull(x), y,x)

In [141]:
df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


## 统计相关 的方法

Series, DataFrame和Panel内置了许多计算统计相关指标的方法。这些方法大致分为两类：
* 返回低维结果，比如sum(),mean(),quantile()
* 返回同原对象同样大小的对象，比如cumsum(), cumprod()

总体来说，这些方法接收一个坐标轴参数:
* Series不需要坐标轴参数
* DataFrame 默认axis=0(index), axis=1(columns)
* Panel 默认axis=1(major), axis=0(items), axis=2(minor)

In [142]:
df

,one,three,two
a,-0.819889,NaN,1.000000
b,1.206005,-0.586788,0.630745
c,-0.620845,-0.080498,-0.227499
d,NaN,-1.495108,-1.590267


In [146]:
df.mean() #axis=0, 计算每一列的平均值

one     -0.078243
three   -0.720798
two     -0.046755
dtype: float64

In [147]:
df.mean(1) #计算每一行的平均值

a    0.090055
b    0.416654
c   -0.309614
d   -1.542688
dtype: float64

所有的这些方法都有skipna参数，含义是计算过程中是否剔除缺失值，skipna默认值为True。

In [148]:
df.sum(0, skipna=False)

one           NaN
three         NaN
two     -0.187021
dtype: float64

In [149]:
df.sum(axis=1, skipna=True)

a    0.180111
b    1.249962
c   -0.928841
d   -3.085375
dtype: float64

**这些函数可以参与算术和广播运算。**

比如：

In [152]:
ts_stand = (df-df.mean())/df.std()

In [153]:
ts_stand.std()

one      1.0
three    1.0
two      1.0
dtype: float64

In [154]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)

In [155]:
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

注意cumsum() cumprod()方法 保留NA值的位置。

In [156]:
df.cumsum()

,one,three,two
a,-0.819889,NaN,1.000000
b,0.386116,-0.586788,1.630745
c,-0.234729,-0.667286,1.403247
d,NaN,-2.162394,-0.187021


下面列出常用的方法及其描述。提醒每一个方法都有一个level参数用于具有层次索引的对象。




|    方法     | 描述  | 
| ------------- |:-------------:| 
| count      | 沿着坐标轴统计非空的行数| 
| sum      | 沿着坐标轴取加和|   
| mean | 沿着坐标轴求均值|    
|mad|沿着坐标轴计算平均绝对偏差|
|median|沿着坐标轴计算中位数|
|min|沿着坐标轴取最小值|
|max|沿着坐标轴取最大值|
|mode|沿着坐标轴取众数|
|abs|计算每一个值的绝对值|
|prod|沿着坐标轴求乘积|
|std|沿着坐标轴计算标准差|
|var|沿着坐标轴计算无偏方差|
|sem|沿着坐标轴计算标准差|
|skew|沿着坐标轴计算样本偏斜|
|kurt|沿着坐标轴计算样本峰度|
|quantile|沿着坐标轴计算样本分位数，单位%|
|cumsum|沿着坐标轴计算累加和|
|cumprod|沿着坐标轴计算累积乘|
|cummax|沿着坐标轴计算累计最大|
|cummin|沿着坐标轴计算累计最小|


**Note**：所有需要沿着坐标轴计算的方法，默认axis=0，即将方法应用到每一列数据上。

Series还有一个nunique()方法返回非空数值 组成的集合的大小。

In [32]:
series = pd.Series(np.random.randn(500))
series[20:500]=np.nan

In [33]:
series[10:20]=5

In [35]:
series.nunique()

11

In [36]:
series

0      0.424958
1      0.167252
2     -1.667684
3     -1.375883
4     -0.272095
5     -1.118394
6     -1.478117
7      0.779770
8     -0.824979
9      0.356511
10     5.000000
11     5.000000
12     5.000000
13     5.000000
14     5.000000
15     5.000000
16     5.000000
17     5.000000
18     5.000000
19     5.000000
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
         ...   
470         NaN
471         NaN
472         NaN
473         NaN
474         NaN
475         NaN
476         NaN
477         NaN
478         NaN
479         NaN
480         NaN
481         NaN
482         NaN
483         NaN
484         NaN
485         NaN
486         NaN
487         NaN
488         NaN
489         NaN
490         NaN
491         NaN
492         NaN
493         NaN
494         NaN
495         NaN
496         NaN
497         NaN
498         NaN
499         NaN
dtype: float64

### descrieb()， 数据摘要

describe()方法非常有用，它计算数据的各种常用统计指标(比如平均值、标准差等)，计算时不包括NA。拿到数据首先要有大概的了解，使用describe()方法就对了。

In [37]:
series = pd.Series(np.random.randn(1000))
series[::2]=np.nan

In [38]:
series.describe()

count    500.000000
mean       0.024341
std        0.987292
min       -2.371843
25%       -0.724967
50%        0.011554
75%        0.682236
max        2.763247
dtype: float64

In [39]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=['a', 'b', 'c', 'd', 'e'])

In [40]:
frame.ix[::2]=np.nan

In [41]:
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,-0.030325,0.001974,-0.060932,0.018964,0.027747
std,0.999444,1.027741,0.976023,0.939193,1.108105
min,-3.038502,-3.152980,-3.343954,-2.478286,-3.260736
25%,-0.722298,-0.654264,-0.698861,-0.632413,-0.705261
50%,-0.009268,-0.001924,-0.064390,0.019038,0.042032
75%,0.624942,0.709595,0.550053,0.615007,0.792178
max,2.825107,2.519475,2.820095,3.619928,3.826766


默认describe()只包含25%, 50%, 75%, 也可以通过percentiles参数进行指定。

In [42]:
series.describe(percentiles=[.05, .25, .75, .95])

count    500.000000
mean       0.024341
std        0.987292
min       -2.371843
5%        -1.474114
25%       -0.724967
50%        0.011554
75%        0.682236
95%        1.693779
max        2.763247
dtype: float64

如果Series内数据是非数值类型，describe()也能给出一定的统计结果

In [43]:
s = pd.Series(['a', 'a', 'b', 'b', 'a', 'a', np.nan, 'c', 'd', 'a'])

In [70]:
s.describe()

count     9
unique    4
top       a
freq      5
dtype: object

如果DataFrame对象有的列是数值类型，有的列不是数值类型，describe()仅对数值类型的列进行计算。

In [47]:
frame = pd.DataFrame({'a':['Yes', 'Yes', 'NO', 'No'], 'b':range(4)})

In [48]:
frame.describe()

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


如果非要知道非数值列的统计指标呢？describe提供了include参数，取值范围{'object', 'number', 'all'}。

看一下例子, 注意'object'和'number'都是在列表中，而'all'不需要放在列表中：

In [57]:
frame.describe(include=['object']) #只对非数值列进行统计计算

,a
count,4
unique,3
top,Yes
freq,2


In [58]:
frame.describe(include=['number'])

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


In [59]:
frame.describe(include='all')#'all'不是列表

,a,b
count,4,4.000000
unique,3,NaN
top,Yes,NaN
freq,2,NaN
mean,NaN,1.500000
std,NaN,1.290994
min,NaN,0.000000
25%,NaN,0.750000
50%,NaN,1.500000
75%,NaN,2.250000


### 最大/最小值对应的索引值

Series和DataFrame内置的idxmin() idxmax()方法求得 最小值、最大值对应的索引值，看一下例子：

In [60]:
s1 = pd.Series(np.random.randn(5))
s1

0    0.619835
1   -0.296405
2    0.598713
3    1.735420
4    0.072205
dtype: float64

In [62]:
s1.idxmin(), s1.idxmax() #最小值：-0.296405, 最大值：1.735420

(1, 3)

In [63]:
df1 = pd.DataFrame(np.random.randn(5,3), columns=list('ABC'))
df1

,A,B,C
0,-0.403992,-0.267194,0.362908
1,0.509844,-0.191461,-0.212362
2,1.012935,-0.180596,0.050764
3,1.249444,0.082952,1.061835
4,-1.861369,0.674862,-0.084830


In [66]:
df1.idxmin(axis=0)

A    4
B    0
C    1
dtype: int64

In [67]:
df1.idxmax(axis=1)

0    C
1    A
2    A
3    A
4    B
dtype: object

如果多个数值都是最大值或最小值，idxmax() idxmin()返回最大值、最小值第一次出现对应的索引值

In [68]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=['A'], index=list('edcba'))
df3

,A
e,2.0
d,1.0
c,1.0
b,3.0
a,NaN


In [69]:
df3['A'].idxmin()

'd'

**实际上，idxmin和idxmax就是NumPy中的argmin和argmax**。

###   value_counts() 数值计数

value_counts()计算一维度数据结构的直方图。

In [71]:
data = np.random.randint(0, 7, size=50)
data

array([0, 3, 4, 1, 5, 5, 2, 5, 0, 3, 0, 2, 3, 0, 5, 5, 4, 1, 1, 1, 6, 4, 0,
       6, 0, 2, 1, 4, 1, 1, 0, 6, 3, 0, 5, 2, 3, 3, 0, 4, 0, 6, 1, 2, 1, 5,
       1, 1, 5, 0])

In [72]:
s = pd.Series(data)

In [73]:
s.value_counts()

1    11
0    11
5     8
3     6
4     5
2     5
6     4
dtype: int64

In [75]:
pd.value_counts(data) #也是全局方法

1    11
0    11
5     8
3     6
4     5
2     5
6     4
dtype: int64

虽然前面介绍过mode()方法了，看两个例子吧：

In [76]:
s5 = pd.Series([1,1,3,3,3,5,5,7,7,7])

In [77]:
s5.mode()

0    3
1    7
dtype: int64

In [83]:
df5 = pd.DataFrame({"A": np.random.randint(0, 7, size=50),
                    "B": np.random.randint(-10, 15, size=50)})

In [106]:
df5

,A,B
0,1,-3
1,1,6
2,0,-8
3,0,4
4,1,-7
5,1,14
6,0,0
7,3,10
8,6,-7
9,3,12


In [107]:
df5.mode()

,A,B
0,1.0,-8
1,NaN,-5
2,NaN,-3
3,NaN,4


### 区间离散化

cut() qcut()方法可以对连续数据进行离散化

In [108]:
arr = np.random.randn(20)

In [109]:
factor = pd.cut(arr, 4)

In [110]:
factor

[(0.143, 1.156], (0.143, 1.156], (-1.886, -0.869], (1.156, 2.168], (0.143, 1.156], ..., (1.156, 2.168], (-0.869, 0.143], (0.143, 1.156], (0.143, 1.156], (-0.869, 0.143]]
Length: 20
Categories (4, object): [(-1.886, -0.869] < (-0.869, 0.143] < (0.143, 1.156] < (1.156, 2.168]]

In [111]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5]) #输入 离散区间

In [112]:
factor

[(0, 1], (0, 1], (-5, -1], (1, 5], (0, 1], ..., (1, 5], (-1, 0], (0, 1], (0, 1], (-1, 0]]
Length: 20
Categories (4, object): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

qcut()方法计算样本的分位数，比如我们可以将正态分布的数据 进行四分位数离散化：

In [113]:
arr = np.random.randn(30)

In [114]:
factor = pd.qcut(arr, [0, .25, .5, .75, 1])

In [115]:
factor

[(-0.0783, 0.444], (-0.0783, 0.444], (-0.799, -0.0783], [-2.71, -0.799], (-0.799, -0.0783], ..., [-2.71, -0.799], (-0.799, -0.0783], (0.444, 1.963], (-0.0783, 0.444], (-0.799, -0.0783]]
Length: 30
Categories (4, object): [[-2.71, -0.799] < (-0.799, -0.0783] < (-0.0783, 0.444] < (0.444, 1.963]]

In [116]:
pd.value_counts(factor)

(0.444, 1.963]       8
[-2.71, -0.799]      8
(-0.0783, 0.444]     7
(-0.799, -0.0783]    7
dtype: int64

离散区间也可以用极限定义

In [117]:
arr = np.random.randn(20)

In [118]:
factor = pd.cut(arr, [-np.inf, 0, np.inf])

In [119]:
factor

[(-inf, 0], (-inf, 0], (0, inf], (-inf, 0], (0, inf], ..., (-inf, 0], (-inf, 0], (0, inf], (-inf, 0], (0, inf]]
Length: 20
Categories (2, object): [(-inf, 0] < (0, inf]]

## 函数应用 